<a href="https://colab.research.google.com/github/bruno-camara/n_reines_algorithmes_heuristiques/blob/main/TP_Metaheuristiques.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algorithmes métaheuristiques appliqués au problème des n reines

Le but du TP est d'implémenter des heuristiques et des choix de voisinage pour la résolution du problème de n-reines sur un échiquier n*n, n > 3.

Réfléchissez d'abord à la représentation d'une solution (un état) et proposer deux manières differentes de définir la notion d'état voisin

Implémentez d'abord la méthode de l'amélioration continue (hill climbing) dans sa version greedy ou dans sa version améliorer un peu en mettant bien en évidence le possible cas d'arrêt dans une configuration qui n'est pas la solution recherchée. Comparez les deux choix et tirer une conclusion.

Implémentez ensuite le recuit simulé ou une autre meta heuristique. 

## **Problème des n reines:**

Mettre n reines sur un échiquier n × n tel 
que et les reines ne s'attaquent pas entre elles.

## État Voisin

L'état voisin est un état obtenu en appliquant une opération de mouvement à l'état courant. Il existe différentes manières de définir la notion d'état voisin. Voici les états voisin que j'ai choisi:

1. **Changer la position d'une reine choisie au hasard** : Dans cette approche, l'état voisin est obtenu en changeant la position d'une seule reine sur l'échiquier. Pour chaque reine dans l'état courant, on examine chaque case de la même rangée qui n'est pas déjà occupée par une autre reine. Si on trouve une case qui n'est pas menacée par une autre reine, on peut déplacer la reine vers cette case pour obtenir un nouvel état voisin.

2. **Changer la position d'une reine en conflit** : La différence entre cette approche et l'approche antérieure est la manière de choisir la reine. Ici, nous allons choisir la reine qui est dans la même ligne qu'une autre reine.

Ces deux approches permettent de définir des opérations de mouvement pour le problème des n-reines, mais il existe de nombreuses autres manières de définir la notion d'état voisin. 

## Modélisation du problème

- Un échiquier vide sera une matrice remplit de 0
- Une reine sera un 1 dans l'échiquier
- Conflit c'est quand il y a des reines qui s'attaquent entre elles

In [1]:
import numpy as np
from numpy import random
import math

In [2]:
def gen_echiquier(n):
  echiquier = np.zeros((n, n))
  return echiquier

In [3]:
def place_reines_randomly(echiquier):
  n = echiquier.shape[0]
  echiquier = echiquier.flatten()
  echiquier[:n] = 1
  np.random.shuffle(echiquier)
  echiquier = echiquier.reshape((n, n))
  return echiquier

In [4]:
def all_traces(x):
    jj = np.tile(np.arange(x.shape[1]),x.shape[0])
    ii = (np.arange(x.shape[1])+np.arange(x.shape[0])[::-1,None]).ravel()
    z = np.zeros(((x.shape[0]+x.shape[1]-1),x.shape[1]),int)
    z[ii,jj] = x.ravel()
    return z.sum(axis=1)

def conflit(echiquier):
  # Check the number of conflicts
  # There will only have a conflict if sum of lines, columns and diagonals are different from 1
  # A conflict does not represent the exact number of conflits, but a representation of the value
  n = echiquier.shape[0]
  ones = np.ones(n)
  conflit_colonne = echiquier.sum(axis=0)[echiquier.sum(axis=0) > ones].sum()
  conflit_ligne = echiquier.sum(axis=1)[echiquier.sum(axis=1) > ones].sum()
  conflit_diagonales = all_traces(echiquier)[all_traces(echiquier) > ones.max(keepdims=True)].sum()
  conflit_antidiagonales = all_traces(np.fliplr(echiquier))[all_traces(np.fliplr(echiquier)) > ones.max(keepdims=True)].sum()
  #print("# Conflit Colonnes:", conflit_colonne)
  #print("# Conflit Lignes:", conflit_ligne)
  #print("# Conflit Diagonales:", conflit_diagonales + conflit_antidiagonales)
  conflits = conflit_colonne + conflit_ligne + conflit_diagonales + conflit_antidiagonales
  return conflits

## Recherche par Amélioration (Hill Climbing)

A partir d'un état initial toujours prendre le voisin qui est meilleur
d'un point de vue coût.

Le coût dans notre cas, c'est le nombre de conflit.

In [5]:
def check_voisin(echiquier, reine):
  # Params1: echiquier - matrix nxn
  # Params2: reine - tuple (line, column) with reine position
  # While there is a better voisin, we change the etat_actuel

  cout = conflit(echiquier)
  echiquier_better = echiquier.copy()

  # Check space below:
  if (reine[0] == echiquier.shape[0] - 1):
    pass
  else:
    echiquier_aux = echiquier.copy()
    if (echiquier[reine[0] + 1, reine[1]] != 1):
      echiquier_aux[reine] = 0
      echiquier_aux[reine[0] + 1, reine[1]] = 1
      if(conflit(echiquier_aux) < cout):
        cout = conflit(echiquier_aux)
        echiquier_better = echiquier_aux.copy()

  # Check space above
  if (reine[0] == 0):
    pass
  else:
    echiquier_aux = echiquier.copy()
    if (echiquier[reine[0] - 1, reine[1]] != 1):
      echiquier_aux[reine] = 0
      echiquier_aux[reine[0] - 1, reine[1]] = 1
      if(conflit(echiquier_aux) < cout):
        cout = conflit(echiquier_aux)
        echiquier_better = echiquier_aux.copy()

  # Check right space
  if (reine[1] == echiquier.shape[1] - 1):
    pass
  else:
    echiquier_aux = echiquier.copy()
    if (echiquier[reine[0], reine[1] + 1] != 1):
      echiquier_aux[reine] = 0
      echiquier_aux[reine[0], reine[1] + 1] = 1
      if(conflit(echiquier_aux) < cout):
        cout = conflit(echiquier_aux)
        echiquier_better = echiquier_aux.copy()

  # Check left space
  if (reine[1] == 0):
    pass
  else:
    echiquier_aux = echiquier.copy()
    if (echiquier[reine[0], reine[1] - 1] != 1):
      echiquier_aux[reine] = 0
      echiquier_aux[reine[0], reine[1] - 1] = 1
      if (conflit(echiquier_aux) < cout):
        cout = conflit(echiquier_aux)
        echiquier_better = echiquier_aux.copy()
    
  # Check upper left diagonal:
  if (reine[0] == 0 or reine[1] == 0):
    pass
  else:
    echiquier_aux = echiquier.copy()
    if (echiquier[reine[0] - 1, reine[1] - 1] != 1):
      echiquier_aux[reine] = 0
      echiquier_aux[reine[0] - 1, reine[1] - 1] = 1
      if (conflit(echiquier_aux) < cout):
        cout = conflit(echiquier_aux)
        echiquier_better = echiquier_aux.copy()

  # Check upper right diagonal:
  if (reine[0] == 0 or reine[1] == echiquier.shape[1] - 1):
    pass
  else:
    echiquier_aux = echiquier.copy()
    if (echiquier[reine[0] - 1, reine[1] + 1] != 1):
      echiquier_aux[reine] = 0
      echiquier_aux[reine[0] - 1, reine[1] + 1] = 1
      if (conflit(echiquier_aux) < cout):
        cout = conflit(echiquier_aux)
        echiquier_better = echiquier_aux.copy()

  # Check lower left diagonal:
  if (reine[0] == echiquier.shape[0] - 1 or reine[1] == 0):
    pass
  else:
    echiquier_aux = echiquier.copy()
    if (echiquier[reine[0] + 1, reine[1] - 1] != 1):
      echiquier_aux[reine] = 0
      echiquier_aux[reine[0] + 1, reine[1] - 1] = 1
      if (conflit(echiquier_aux) < cout):
        cout = conflit(echiquier_aux)
        echiquier_better = echiquier_aux.copy()

  # Check lower right diagonal:
  if (reine[0] == echiquier.shape[0] - 1 or reine[1] == echiquier.shape[1] - 1):
    pass
  else:
    echiquier_aux = echiquier.copy()
    if (echiquier[reine[0] + 1, reine[1] + 1] != 1):
      echiquier_aux[reine] = 0
      echiquier_aux[reine[0] + 1, reine[1] + 1] = 1
      if (conflit(echiquier_aux) < cout):
        cout = conflit(echiquier_aux)
        echiquier_better = echiquier_aux.copy()
    
  return echiquier_better


In [6]:
def choisir_reine_hasard(echiquier):
  # Param1: echiquier - matrix nxn
  # return: reine - tuple (x, y)
  # Chose a queen randomly

  reines = np.where(echiquier == 1)
  rnd_number = random.randint(echiquier.shape[0])
  reine = (reines[0][rnd_number], reines[1][rnd_number])
  return reine

In [7]:
def choisir_reine(echiquier):
  # Param1: echiquier - matrix nxn
  # return: reine - tuple (x, y)
  # Chose a queen in the same line as another

  if (np.unique(np.where(echiquier == 1)[0], return_counts=True)[1].max() > 1):
    lines, cols = np.where(echiquier == 1)

    rnd_number = random.randint(cols[np.where(lines == lines.max())].shape[0])
    reine = (lines[np.where(lines == lines.max())][rnd_number], cols[np.where(lines == lines.max())][rnd_number])

    return reine
  else:
    return choisir_reine_hasard(echiquier)

In [8]:
def hill_climbing(etat_initial):
  # Param1: etat_initial - echiquier nxn
  # Using the état voisin 1
  etat_actuel = etat_initial
  cout = conflit(etat_actuel)
  i = 0
  couts_anterieurs = np.random.randint(1,100,100)
  stop_cond = False # Stop Condition
  while (~stop_cond and cout != 0):
    etat_actuel = check_voisin(etat_actuel, choisir_reine_hasard(etat_actuel))
    cout = conflit(etat_actuel)
    couts_anterieurs[i%100] = cout
    stop_cond = np.all(couts_anterieurs == couts_anterieurs[0])
    i = i + 1
  if (stop_cond):
    print("Max Local found")
  else:
    print("Max Global found")
  return etat_actuel

In [9]:
def hill_climbing_2(etat_initial):
  # Param1: etat_initial - echiquier nxn
  # Using the état voisin 2
  
  etat_actuel = etat_initial
  cout = conflit(etat_actuel)
  i = 0
  couts_anterieurs = np.random.randint(1,100,100)
  stop_cond = False # Stop Condition
  while (~stop_cond and cout != 0):
    etat_actuel = check_voisin(etat_actuel, choisir_reine(etat_actuel))
    cout = conflit(etat_actuel)
    couts_anterieurs[i%100] = cout
    stop_cond = np.all(couts_anterieurs == couts_anterieurs[0])
    i = i + 1
  if (stop_cond):
    print("Max Local found")
  else:
    print("Max Global found")
  return etat_actuel

In [10]:
def hill_climbing_iterations(etat_initial):
  # Param1: etat_initial - echiquier nxn
  # Hill Climbing stop after a certain number of iterations
  
  etat_actuel = etat_initial
  cout = conflit(etat_actuel)
  i = 0
  while (i < 10000 and cout != 0):
    etat_actuel = check_voisin(etat_actuel, choisir_reine_hasard(etat_actuel))
    cout = conflit(etat_actuel)
    i = i + 1
  if (i == 10000):
    print("Max Local found")
  else:
    print("Max Global found")
  return etat_actuel

In [11]:
# Exemple
echiquier = gen_echiquier(8)
echiquier = place_reines_randomly(echiquier)
print("État Initial\n")
print(echiquier)
print("\n")
print("# Conflits Initiales:", conflit(echiquier))
print("---------------------------------------------------")
print("État Final - État voisin 1\n")
echiquier_resolu = hill_climbing(echiquier)
print(echiquier_resolu)
print("\n")
print("# Conflits Finales:", conflit(echiquier_resolu))
print("---------------------------------------------------")
print("État Final - État voisin 2\n")
echiquier_resolu = hill_climbing_2(echiquier)
print(echiquier_resolu)
print("\n")
print("# Conflits Finales:", conflit(echiquier_resolu))

État Initial

[[0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 1. 0. 1.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0.]]


# Conflits Initiales: 17.0
---------------------------------------------------
État Final - État voisin 1

Max Local found
[[0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0.]]


# Conflits Finales: 4.0
---------------------------------------------------
État Final - État voisin 2

Max Local found
[[0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 1. 0. 1.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]]


# Conflits Finales: 16.0


## Recuit Simulé (Simulated Annealing)

Appelée aussi gradient descendent : avec une probabilité de type
e^(−λt) (décroissante dans le temps) permettre le choix d'un voisin
qui n'est pas meilleur afin d'échapper d'un maximum local

In [12]:
def voisin_aleatoire(echiquier, reine):
  # Params1: echiquier - matrix nxn
  # Params2: reine - tuple (line, column) with reine position
  # Chose a random neighboor

  allowed_movements = []

  # Check space below:
  if (reine[0] != echiquier.shape[0] - 1 and echiquier[reine[0] + 1, reine[1]] != 1):
    allowed_movements.append("below")

  # Check space above
  if (reine[0] != 0 and echiquier[reine[0] - 1, reine[1]] != 1):
    allowed_movements.append("above")

  # Check right space
  if (reine[1] != echiquier.shape[1] - 1 and echiquier[reine[0], reine[1] + 1] != 1):
    allowed_movements.append("right")

  # Check left space
  if (reine[1] != 0 and echiquier[reine[0], reine[1] - 1] != 1):
    allowed_movements.append("left")
    
  # Check upper left diagonal:
  if (reine[0] != 0 and reine[1] != 0 and echiquier[reine[0] - 1, reine[1] - 1] != 1):
    allowed_movements.append("upper_left")

  # Check upper right diagonal:
  if (reine[0] != 0 and reine[1] != echiquier.shape[1] - 1 and echiquier[reine[0] - 1, reine[1] + 1] != 1):
    allowed_movements.append("upper_right")
  
  # Check lower left diagonal:
  if (reine[0] != echiquier.shape[0] - 1 and reine[1] != 0 and echiquier[reine[0] + 1, reine[1] - 1] != 1):
    allowed_movements.append("lower_left")

  # Check lower right diagonal:
  if (reine[0] != echiquier.shape[0] - 1 and reine[1] != echiquier.shape[1] - 1 and echiquier[reine[0] + 1, reine[1] + 1] != 1):
    allowed_movements.append("lower_right")

  # Chose randomly one of the possible movements:
  rnd_number = random.randint(len(allowed_movements))
  echiquier_aux = echiquier.copy()
  choice = allowed_movements[rnd_number]

  if (choice == "below"):
    echiquier_aux[reine] = 0
    echiquier_aux[reine[0] + 1, reine[1]] = 1
  elif (choice == "above"):
    echiquier_aux[reine] = 0
    echiquier_aux[reine[0] - 1, reine[1]] = 1
  elif (choice == "right"):
    echiquier_aux[reine] = 0
    echiquier_aux[reine[0], reine[1] + 1] = 1
  elif (choice == "left"):
    echiquier_aux[reine] = 0
    echiquier_aux[reine[0], reine[1] - 1] = 1
  elif (choice == "upper_left"):
    echiquier_aux[reine] = 0
    echiquier_aux[reine[0] - 1, reine[1] - 1] = 1
  elif (choice == "upper_right"):
    echiquier_aux[reine] = 0
    echiquier_aux[reine[0] - 1, reine[1] + 1] = 1
  elif (choice == "lower_left"):
    echiquier_aux[reine] = 0
    echiquier_aux[reine[0] + 1, reine[1] - 1] = 1
  elif (choice == "lower_right"):
    echiquier_aux[reine] = 0
    echiquier_aux[reine[0] + 1, reine[1] + 1] = 1
    
  return echiquier_aux

In [13]:
def recuit_simule(etat_initial):
  # Param1: etat_initial - echiquier nxn
  # Using état voisin 1

  etat_actuel = etat_initial
  t = 1
  while (t < 10000 and conflit(etat_actuel) != 0):
    suivant = voisin_aleatoire(etat_actuel, choisir_reine_hasard(etat_actuel))
    delta = conflit(suivant) - conflit(etat_actuel);
    if (delta <= 0):
      etat_actuel = suivant;
    else:
      chance = math.exp(-delta*t)
      if(math.floor(random.uniform(0, 1/(1-chance))) == 1):
          etat_actuel = suivant
    t = t + 1
  if (t == 10000):
    print("Max Local found")
  else:
    print("Max Global found")
  return etat_actuel

In [14]:
def recuit_simule_2(etat_initial):
  # Param1: etat_initial - echiquier nxn
  # Using état voisin 2

  etat_actuel = etat_initial
  t = 1
  while (t < 10000 and conflit(etat_actuel) != 0):
    suivant = voisin_aleatoire(etat_actuel, choisir_reine(etat_actuel))
    delta = conflit(suivant) - conflit(etat_actuel);
    if (delta <= 0):
      etat_actuel = suivant;
    else:
      chance = math.exp(-delta*t)
      if(math.floor(random.uniform(0, 1/(1-chance))) == 1):
          etat_actuel = suivant
    t = t + 1
  if (t == 10000):
    print("Max Local found")
  else:
    print("Max Global found")
  return etat_actuel

In [15]:
# Exemple Recuit Simulé
echiquier = gen_echiquier(8)
echiquier = place_reines_randomly(echiquier)
print("État initial\n")
print(echiquier)
print("\n")
print("# Conflits Initiales:", conflit(echiquier))
print("---------------------------------------------------")
print("État final\n")
echiquier_resolu = recuit_simule(echiquier)
print(echiquier_resolu)
print("\n")
print("# Conflits Finales:", conflit(echiquier_resolu))
print("---------------------------------------------------")
print("État final 2\n")
echiquier_resolu = recuit_simule_2(echiquier)
print(echiquier_resolu)
print("\n")
print("# Conflits Finales:", conflit(echiquier_resolu))

État initial

[[0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]]


# Conflits Initiales: 14.0
---------------------------------------------------
État final

Max Global found
[[0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]]


# Conflits Finales: 0.0
---------------------------------------------------
État final 2

Max Local found
[[0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 1.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]]


# Conflits Finales: 12.0


## Mon ressenti

J'ai trouvé très intéressant de voir comment fonctionnent les algorithmes heuristiques. 

Une particularité est que je n'ai pas toujours pu atteindre le résultat optimal, car j'ai souvent atteint le maximum local et non le maximum global.

D'après mes observations, j'ai pu constater que le deuxième algorithme menait plus souvent au résultat optimal, en raison de son mécanisme de déplacement de la reine même si le coût ne change pas, évitant ainsi les maxima locaux.

J'ai également pu constater que l'état voisin n'influence pas beaucoup le résultat final, puisque les deux approches sont très similaires.

## Utilities

### Create README

In [20]:
%%shell
echo -e "# Comment Déployer\n" > README.md
echo -e "Language: $(python --version)\n" >> README.md
echo -e "J'ai utilisé google colab pour ce projet, mais avec n'importe quel compilateur JupyterNotebook c'est possible d'exécuter ce notebook.\n" >> README.md
echo -e "Les spécifications du projet et mon ressenti sont écrit au notebook" >> README.md
cat README.md

# Comment Déployer

Language: Python 3.9.16

J'ai utilisé google colab pour ce projet, mais avec n'importe quel compilateur JupyterNotebook c'est possible d'exécuter ce notebook.

Les spécifications du projet et mon ressenti sont écrit au notebook


### Transform Notebook to HTML

To do so:
1. download the notebook
2. upload it again in the colab
3. execute the cell below
4. check the html file in the files of the colab

In [17]:
%%shell
jupyter nbconvert --to html /TP_Metaheuristiques.ipynb

[NbConvertApp] Converting notebook /TP_Metaheuristiques.ipynb to html
[NbConvertApp] Writing 669007 bytes to /TP_Metaheuristiques.html
